# Đồ án 2: Image Processing

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

## Thay đổi độ sáng

In [ ]:
def adjust_brightness(img, brightness):
    return np.uint8(np.clip(img + np.array([brightness], dtype=np.int16), 0, 255))

## Thay đổi độ tương phản

In [ ]:
def adjust_contrast(img, contrast):
    contrast = np.clip(float(contrast), -255, 255)
    factor = (259 * (contrast + 255)) / (255 * (259 - contrast))
    return np.uint8(np.clip(factor * (img.astype(float) - 128) + 128, 0, 255))

## Chuyển đổi ảnh RGB thành ảnh xám

In [ ]:
def to_grayscale(img):
    return np.uint8(np.dot(img[... , :3], [0.299 , 0.587, 0.114]))

## Lật ảnh ngang - dọc

In [ ]:
def flip(img, direction):
    return np.flipud(img) if direction == 'vertical' else np.fliplr(img)

## Chồng hai ảnh cùng kích thước

In [ ]:
def blend(img_1, img_2, alpha):
    return np.uint8(alpha * img_1.astype(float) + (1-alpha) * img_2.astype(float))

## Làm mờ ảnh

In [ ]:
def Gaussian_func(x, sigma):
    return np.array(1 / (np.sqrt(2 * np.pi) * sigma) * (np.exp(-np.power(x / sigma, 2) / 2)))

In [ ]:
def calc_Gaussian_kernel(kernel_size, sigma):
    kernel_1d = np.linspace(-(kernel_size // 2), kernel_size // 2, num=kernel_size)
    kernel_1d = Gaussian_func(kernel_1d, sigma)
    kernel_2d = np.outer(kernel_1d.T, kernel_1d.T)
    kernel_2d *= 1.0 / np.sum(kernel_2d)
    return kernel_2d

In [ ]:
def convolution(img, kernel):
    result = np.zeros(img.shape)

    image_row, image_col, image_dim = img.shape
    kernel_row, kernel_col = kernel.shape 
    pad_height = (kernel_row - 1) // 2
    pad_width = (kernel_col - 1) // 2
 
    padded_image = np.zeros((image_row + (2 * pad_height), image_col + (2 * pad_width), image_dim))
 
    padded_image[pad_height:padded_image.shape[0] - pad_height, pad_width:padded_image.shape[1] - pad_width] = img
    
    for channel in range(image_dim):
        for row in range(image_row):
            for col in range(image_col):
                result[row, col, channel] = np.sum(kernel * padded_image[row:row + kernel_row, col:col + kernel_col, channel])
    
    return result.astype(np.uint8)

In [ ]:
def Gaussian_blur(img, kernel_size):
    kernel = calc_Gaussian_kernel(kernel_size, sigma=(kernel_size-1)/6)
    return convolution(img, kernel)

## Test các hàm với ảnh cụ thể

### Mở ảnh và đưa ảnh về `np.ndarray`

In [ ]:
input_file = 'tiger.png'
image = Image.open(input_file)
image = np.array(image)

### Thay đổi độ sáng

In [ ]:
# Brightness: -255 to 255
brightness = 0
result = adjust_brightness(image, brightness)

In [ ]:
plt.imshow(result)

In [ ]:
output_file = input_file.split('.')[0] + '_' + str(brightness) + '_brightness' + '.png'
Image.fromarray(result).save(output_file)

### Thay đổi độ tương phản

In [ ]:
# Contrast: -255 to 255
contrast = 0
result = adjust_contrast(image, contrast)

In [ ]:
plt.imshow(result)

In [ ]:
output_file = input_file.split('.')[0] + '_' + str(contrast) + '_contrast' + '.png'
Image.fromarray(result).save(output_file)

### Chuyển thành ảnh xám

In [ ]:
result = to_grayscale(image)

In [ ]:
plt.imshow(result, cmap='gray')

In [ ]:
output_file = input_file.split('.')[0] + '_grayscale' + '.png'
Image.fromarray(result).save(output_file, cmap='gray')

### Lật ảnh

In [ ]:
# Direction: 'vertical' or 'horizontal'
direction = 'horizontal'
result = flip(image, direction)

In [ ]:
plt.imshow(result)

In [ ]:
output_file = input_file.split('.')[0] + '_flip_' + direction + '.png'
Image.fromarray(result).save(output_file)

### Chồng ảnh

In [ ]:
input_file_2 = 'tiger.png'
image_2 = Image.open(input_file)
image_2 = np.array(image_2)

In [ ]:
gray_image_1 = to_grayscale(image)
gray_image_2 = flip(to_grayscale(image_2), 'horizontal')

In [ ]:
# Alpha: 0.0 to 1.0
alpha = 0.5
result = blend(gray_image_1, gray_image_2, alpha) 

In [ ]:
plt.imshow(result, cmap='gray')

In [ ]:
output_file = input_file.split('.')[0] + '_' + input_file_2.split('.')[0] + '_' + str(alpha) + '_blend' + '.png'
Image.fromarray(result).save(output_file, cmap='gray')

### Làm mờ ảnh

In [ ]:
# Kernel size: 1 to +inf, is even
kernel_size = 15
result = Gaussian_blur(image, kernel_size)

In [ ]:
plt.imshow(result)

In [ ]:
output_file = input_file.split('.')[0] + '_' + str(kernel_size) + '_blur' + '.png'
Image.fromarray(result).save(output_file)